In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import csv
import json
import pandas as pd
import requests
import re
print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# Spoofer code
try:

    import sys
    import os

    from fp.fp import FreeProxy
    from fake_useragent import UserAgent
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver import Chrome
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    import time
    print('all module are loaded ')

except Exception as e:

    print("Error ->>>: {} ".format(e))


class Spoofer(object):

    def __init__(self, country_id=['US'], rand=True, anonym=True):
        self.country_id = country_id
        self.rand = rand
        self.anonym = anonym
        self.userAgent, self.ip = self.get()

    def get(self):
        ua = UserAgent()
        proxy = FreeProxy(country_id=self.country_id, rand=self.rand, anonym=self.anonym).get()
        ip = proxy.split("://")[1]
        return ua.random, ip


class DriverOptions(object):

    def __init__(self):

        self.options = Options()
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--start-maximized')
        self.options.add_argument('--start-fullscreen')
        self.options.add_argument('--single-process')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument("--incognito")
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_experimental_option('useAutomationExtension', False)
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_argument("disable-infobars")

        self.helperSpoofer = Spoofer()

        self.options.add_argument('user-agent={}'.format(self.helperSpoofer.userAgent))
        self.options.add_argument('--proxy-server=%s' % self.helperSpoofer.ip)


class WebDriver(DriverOptions):

    def __init__(self, path=''):
        DriverOptions.__init__(self)
        self.driver_instance = self.get_driver()

    def get_driver(self):

        print("""
        IP:{}
        UserAgent: {}
        """.format(self.helperSpoofer.ip, self.helperSpoofer.userAgent))

        PROXY = self.helperSpoofer.ip
        webdriver.DesiredCapabilities.CHROME['proxy'] = {
            "httpProxy":PROXY,
            "ftpProxy":PROXY,
            "sslProxy":PROXY,
            "noProxy":None,
            "proxyType":"MANUAL",
            "autodetect":False
        }
        webdriver.DesiredCapabilities.CHROME['acceptSslCerts'] = True

        path = os.path.join(os.getcwd(), '/chromedriver.exe')

        driver = webdriver.Chrome()
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source":
                "const newProto = navigator.__proto__;"
                "delete newProto.webdriver;"
                "navigator.__proto__ = newProto;"
        })

        return driver

all module are loaded 


In [3]:
# Function to create a csv by giving the field name, its contents, the directory name and filename

def create_csv(fields, rows, dirname, filename):
    with open(dirname+filename, 'w', encoding='UTF8', newline='') as csvfile: 
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields) 
        csvwriter.writerows(rows)
    print(f"Saved csv with name {filename}")

In [5]:
# Create anti bot detection web driver

while True:
    try:
        driver = WebDriver()
        break
    except: # If Driver creation fails try again
        print("Driver creation failed.....Trying again")
        continue
print("Driver Successfully created")


        IP:20.47.108.204:8888
        UserAgent: Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20100101 Firefox/21.0
        
Driver Successfully created


In [4]:
# Function to extract url of every attraction from the given page url
def get_attr_urls(url):

    # Create Driver instance and open the webpage through selenium
    driverinstance = driver.driver_instance
    driverinstance.get(url)

    # Click on the "SHOW MORE" button 20 times to load enough hotels 
    for i in range(10):
        sleep_time = 5 + 5*random.random()
        time.sleep(sleep_time)
        driverinstance.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        try:
            button = driverinstance.find_element_by_xpath('//div[@class="uitk-spacing uitk-type-center uitk-spacing-margin-blockstart-two uitk-spacing-margin-blockend-four"]/button')
            button.click()
        except:
            print("Button not Found, ending search")
            break

    # Create BeautifulSoup instance and search for each hotel on the page to get it's link
    soup = str(BeautifulSoup(driverinstance.page_source, 'html.parser'))
    soup = BeautifulSoup(soup, 'lxml')

    # Getting the url of each attraction and returning it as a list
    sec = soup.find('section', class_="uitk-layout-grid uitk-layout-grid-gap-three uitk-layout-grid-align-content-space-between uitk-layout-grid-columns-small-2 uitk-layout-grid-columns-medium-3 uitk-layout-grid-columns-large-3") 
    card = sec.find_all('div' , attrs= {"data-testid":"activity-tile-card"})
    attraction_url_list=[]
    for i in card:
        attr_url = "https://www.expedia.com"+i.find('a', class_ = "uitk-card-link").attrs['href']
        attraction_url_list.append(attr_url)
    return attraction_url_list

In [7]:
# Open the text file containing links to page with attractions of each category
with open('Testing/Project-Dataset/jaipur_category_urls.txt', newline='') as f:
    reader = csv.reader(f)
    category_full_urllist = list(reader)
length_df = int(len(category_full_urllist))

# Iterating through all the category links to get links of each attraction
attraction_urls = []
for iter_val in range(length_df):
    url = str(category_full_urllist[iter_val])[2:-2]
    attraction_urls+= get_attr_urls(url)
print(len(attraction_urls))

Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
Button not Found, ending search
1847


In [8]:
# Removing Duplicate attractions

res = []
for i in attraction_urls:
    if i not in res:
        res.append(i)
print(f'Number of Attractions extracted: {len(res)}')

Number of Attractions extracted: 1722


In [9]:
# Save all the links in a text file

with open("Testing/Project-Dataset/jaipur_attraction_urls.txt", 'w') as output:
    for row in res:
        output.write(str(row) + '\n')
print("Attraction links saved in text file successfully")

Attraction links saved in text file successfully
